In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 
import requests
from json import load
import json

ModuleNotFoundError: No module named 'psycopg2'

## Open JSON file - Download file from link in Readme or Resources

In [2]:
json_file_ghg ="global-greenhouse-gas-emissions.json"

In [71]:
gghg_file_df = pd.read_json(json_file_ghg)
gghg_file_df.head()

,datasetid,recordid,fields
0,global-greenhouse-gas-emissions,f54a80719f59eb970175add568e6ce7f33bfdd66,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."
1,global-greenhouse-gas-emissions,706d949e60cc90591ca4f628818cf8d9b4f0c577,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."
2,global-greenhouse-gas-emissions,939cf5d648c2b37258f0545860f9159de04babe3,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."
3,global-greenhouse-gas-emissions,795477055ac695d8e8ddcd7f6dba993c1f3253bb,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."
4,global-greenhouse-gas-emissions,a06fcfc69e16bcb8bbc892bc094f2cf12f54a77c,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."


In [72]:
gghg_fields_df = gghg_file_df.loc[:, "fields"]
gghg_fields_df.head()

0    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
1    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
2    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
3    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
4    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
Name: fields, dtype: object

## Create a DataFrame

In [73]:
norm_gghg_fields = pd.json_normalize(gghg_fields_df, max_level = 2)
norm_gghg_fields.head()

,ipcc_annex,ipcc,name,value,year,compound,iso_a3,world_region,ipcc_description
0,Annex_I,6D,Bulgaria,0.025106,1985,N2O,BGR,Central Europe,Other waste handling
1,Annex_I,6D,Bulgaria,0.025106,2011,N2O,BGR,Central Europe,Other waste handling
2,Annex_I,6D,Bulgaria,0.025106,1990,N2O,BGR,Central Europe,Other waste handling
3,Annex_I,6D,Bulgaria,0.025106,1992,N2O,BGR,Central Europe,Other waste handling
4,Annex_I,6D,Bulgaria,0.025106,1995,N2O,BGR,Central Europe,Other waste handling


### Selecting 50 Countries Representating Global Emissions DataBase

In [76]:
counrty_list = ['United States', 'Mexico', 'Brazil', 'Columbia', 'Venezuela','Chile','Peru','India','Bangladesh','Sri Lanka',
                'Myanmar', 'Pakistan', 'Russia', 'Ukraine', 'Kazakhstan', 'Azerbaijan', 'Armenia', 'Afghanistan', 'Uzbekistan',
                'South Africa', 'Nigeria', 'Ghana', 'Morocco', 'Egypt', 'Kenya', 'China', 'Japan', 'Hong Kong', 'Singapore', 
                'Thailand', 'Malaysia', 'Korea', 'Viet Nam', 'United Kingdom', 'France', 'Germany', 'Norway', 'Italy', 
                'Spain', 'Portugal', 'Sweden', 'Turkey', 'Greece', 'Saudi Arabia', 'Bahrain', 'Syrian Arab Republic', 
                'Iraq', 'Qatar', 'United Arab Emirates', 'Kuwait']

In [83]:
gghg_clist = norm_gghg_sort.loc[norm_gghg_sort["name"].isin(counrty_list)]
gghg_clist = gghg_clist.dropna(how = 'any')
gghg_clist.reset_index (drop = True, inplace = True)
gghg_clist

,ipcc_annex,ipcc,name,value,year,compound,iso_a3,world_region,ipcc_description
0,Non-Annex_I,1.A.4,Afghanistan,2.828681e-02,1995,N2O,AFG,India +,Other Sectors
1,Non-Annex_I,3D,Afghanistan,3.712901e-03,1974,N2O,AFG,India +,Solvent and other product use: other
2,Non-Annex_I,1B2,Afghanistan,1.307460e-05,2001,N2O,AFG,India +,Fugitive emissions from oil and gas
3,Non-Annex_I,1B2,Afghanistan,9.562420e-06,2002,N2O,AFG,India +,Fugitive emissions from oil and gas
4,Non-Annex_I,1B2,Afghanistan,9.261660e-09,1971,N2O,AFG,India +,Fugitive emissions from oil and gas
...,...,...,...,...,...,...,...,...,...
253350,Non-Annex_I,2.D,Viet Nam,8.337718e+01,1986,CO2_excl_short-cycle_org_C,VNM,Southeastern Asia,Non-Energy Products from Fuels and Solvent Use
253351,Non-Annex_I,2.D,Viet Nam,7.305094e+01,1980,CO2_excl_short-cycle_org_C,VNM,Southeastern Asia,Non-Energy Products from Fuels and Solvent Use
253352,Non-Annex_I,2.C,Viet Nam,1.256000e+01,1997,CO2_excl_short-cycle_org_C,VNM,Southeastern Asia,Metal Industry
253353,Non-Annex_I,4.D,Viet Nam,4.649097e+02,2006,CH4,VNM,Southeastern Asia,Wastewater Treatment and Discharge


### Delete rows to simplify Quering and plotting - The DF would provide a list of emissions per country for specific sector and year

In [84]:
del gghg_clist['ipcc_annex']
del gghg_clist['ipcc']
del gghg_clist['world_region']
del gghg_clist['iso_a3']
gghg_clist

,name,value,year,compound,ipcc_description
0,Afghanistan,2.828681e-02,1995,N2O,Other Sectors
1,Afghanistan,3.712901e-03,1974,N2O,Solvent and other product use: other
2,Afghanistan,1.307460e-05,2001,N2O,Fugitive emissions from oil and gas
3,Afghanistan,9.562420e-06,2002,N2O,Fugitive emissions from oil and gas
4,Afghanistan,9.261660e-09,1971,N2O,Fugitive emissions from oil and gas
...,...,...,...,...,...
253350,Viet Nam,8.337718e+01,1986,CO2_excl_short-cycle_org_C,Non-Energy Products from Fuels and Solvent Use
253351,Viet Nam,7.305094e+01,1980,CO2_excl_short-cycle_org_C,Non-Energy Products from Fuels and Solvent Use
253352,Viet Nam,1.256000e+01,1997,CO2_excl_short-cycle_org_C,Metal Industry
253353,Viet Nam,4.649097e+02,2006,CH4,Wastewater Treatment and Discharge


### List of Vairous GH Gases in Global Dataset 

In [85]:
glob_emis_gases = gghg_clist['compound'].value_counts()
glob_emis_gases_df = pd.DataFrame(glob_emis_gases)
glob_emis_gases_df

,compound
CO2_excl_short-cycle_org_C,83043
N2O,80320
CH4,75985
CO2_org_short-cycle_C,14007


### Rename Organize and sort DF 

In [87]:
Rename_gghg_clist = gghg_clist.rename(columns = {"name": "Country", "value":"Global_Emission_ktonnes", "compound":"GH-gases", "ipcc_description": "Industry_Sector" })
organized_gghg_df = Rename_gghg_clist [["year", "Country", "GH-gases", "Industry_Sector", "Global_Emission_ktonnes"]]
organized_gghg_df = organized_gghg_df.sort_values('year')
organized_gghg_df.reset_index(drop=True, inplace=True)
organized_gghg_df

,year,Country,GH-gases,Industry_Sector,Global_Emission_ktonnes
0,1970,United States,N2O,Chemical Industry,328.770000
1,1970,India,N2O,Emissions from biomass burning,4.098862
2,1970,Ghana,CH4,Petroleum Refining - Manufacture of Solid Fuel...,0.000818
3,1970,Germany,CO2_excl_short-cycle_org_C,Chemical Industry,12934.666900
4,1970,Brazil,N2O,Other Energy Industries,0.042225
...,...,...,...,...,...
253350,2018,Japan,CO2_excl_short-cycle_org_C,Other direct soil emissions,11850.534733
253351,2018,Chile,CO2_excl_short-cycle_org_C,Road transportation,23776.018114
253352,2018,Saudi Arabia,CO2_excl_short-cycle_org_C,Road transportation,127042.268102
253353,2018,Saudi Arabia,CO2_excl_short-cycle_org_C,Domestic aviation,2753.270825


In [88]:
organized_gghg_df.describe()

,Global_Emission_ktonnes
count,2.533550e+05
mean,8.154872e+03
std,7.899003e+04
min,-2.000000e-03
25%,2.585709e-01
50%,8.515735e+00
75%,5.167879e+02
max,4.585523e+06


## Connect to Local SQL DataBase

In [89]:
from config import password

In [90]:
protocol = 'postgresql'
username = 'postgres'
host = 'localhost'
port = 5432
database_name = 'Global_Emissions'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
# connection = engine.connect()

In [91]:
engine.table_names()

['Global_Greenhouse_Emissions',
 'Global_Agri_Greenhouse_Emissions',
 'Global_Agri_Percent_Emissions']

In [92]:
organized_gghg_df.to_sql(name='Global_Greenhouse_Emissions', con=engine, if_exists='append', index=False)